In [1]:
%load_ext autoreload
%load_ext lab_black
%autoreload 2

In [2]:
import sys

sys.path.append("..")

---

In [7]:
import pandas as pd

from funcs.utils import find_project_root

---

# Association

In [44]:
root = find_project_root()
data_dir = root.parent / "data"
assert data_dir.exists()
ebi_dir = data_dir / "gwas-catalog"

In [30]:
df_path = ebi_dir / "gwas_catalog_v1.0.2-studies_r2020-11-03.tsv"
assert df_path.exists()

In [8]:
df0 = pd.read_csv(df_path, sep="\t")
print(df0.info())
df0.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10233 entries, 0 to 10232
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   DATE ADDED TO CATALOG       10233 non-null  object
 1   PUBMEDID                    10233 non-null  int64 
 2   FIRST AUTHOR                10233 non-null  object
 3   DATE                        10233 non-null  object
 4   JOURNAL                     10233 non-null  object
 5   LINK                        10233 non-null  object
 6   STUDY                       10233 non-null  object
 7   DISEASE/TRAIT               10233 non-null  object
 8   INITIAL SAMPLE SIZE         10233 non-null  object
 9   REPLICATION SAMPLE SIZE     3826 non-null   object
 10  PLATFORM [SNPS PASSING QC]  10233 non-null  object
 11  ASSOCIATION COUNT           10233 non-null  int64 
 12  MAPPED_TRAIT                10221 non-null  object
 13  MAPPED_TRAIT_URI            10221 non-null  ob

,DATE ADDED TO CATALOG,PUBMEDID,FIRST AUTHOR,DATE,JOURNAL,LINK,STUDY,DISEASE/TRAIT,INITIAL SAMPLE SIZE,REPLICATION SAMPLE SIZE,PLATFORM [SNPS PASSING QC],ASSOCIATION COUNT,MAPPED_TRAIT,MAPPED_TRAIT_URI,STUDY ACCESSION,GENOTYPING TECHNOLOGY
0,2012-10-19,21102463,Franke A,2010-11-21,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/21102463,Genome-wide meta-analysis increases to 71 the ...,Crohn's disease,"6,333 European ancestry cases, 15,056 European...","15,694 European ancestry cases, 14,026 Europea...","Affymetrix, Illumina [953241] (imputed)",71,Crohn's disease,http://www.ebi.ac.uk/efo/EFO_0000384,GCST000879,Genome-wide genotyping array
1,2011-01-13,21051598,Pereyra F,2010-11-04,Science,www.ncbi.nlm.nih.gov/pubmed/21051598,The major genetic determinants of HIV-1 contro...,HIV-1 control,"516 European cases, 1,196 European controls, 3...",NaN,Illumina [1384048] (imputed),8,HIV-1 infection,http://www.ebi.ac.uk/efo/EFO_0000180,GCST000863,Genome-wide genotyping array
2,2010-09-15,20700443,Meyer TE,2010-08-05,PLoS Genet,www.ncbi.nlm.nih.gov/pubmed/20700443,Genome-wide association studies of serum magne...,Magnesium levels,"15,366 European ancestry individuals","8,463 European ancestry individuals","Affymetrix, Illumina [~ 2500000] (imputed)",9,magnesium measurement,http://www.ebi.ac.uk/efo/EFO_0004845,GCST000756,Genome-wide genotyping array
3,2009-01-12,19060906,Kathiresan S,2008-12-07,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/19060906,Common variants at 30 loci contribute to polyg...,HDL cholesterol,"19,840 European ancestry individuals","Up to 20,623 European ancestry individuals","Affymetrix, Illumina [~ 2600000] (imputed)",14,high density lipoprotein cholesterol measurement,http://www.ebi.ac.uk/efo/EFO_0004612,GCST000290,Genome-wide genotyping array
4,2009-01-14,19060906,Kathiresan S,2008-12-07,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/19060906,Common variants at 30 loci contribute to polyg...,LDL cholesterol,"19,840 European ancestry individuals","Up to 20,623 European ancestry individuals","Affymetrix, Illumina [~ 2600000] (imputed)",11,low density lipoprotein cholesterol measurement,http://www.ebi.ac.uk/efo/EFO_0004611,GCST000287,Genome-wide genotyping array


In [22]:
df = (
    df0.assign(
        trait_id=lambda df: [
            str(idx) + "-" + str(pubmedid)
            for idx, pubmedid in enumerate(df["PUBMEDID"].tolist())
        ]
    )
    .rename(
        columns={
            "DISEASE/TRAIT": "trait_name",
            "MAPPED_TRAIT": "efo_term",
            "MAPPED_TRAIT_URI": "efo_id",
        }
    )[["trait_id", "trait_name", "efo_term", "efo_id"]]
    .dropna()
    .drop_duplicates()
)
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10221 entries, 0 to 10232
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   trait_id    10221 non-null  object
 1   trait_name  10221 non-null  object
 2   efo_term    10221 non-null  object
 3   efo_id      10221 non-null  object
dtypes: object(4)
memory usage: 399.3+ KB
None


,trait_id,trait_name,efo_term,efo_id
0,0-21102463,Crohn's disease,Crohn's disease,http://www.ebi.ac.uk/efo/EFO_0000384
1,1-21051598,HIV-1 control,HIV-1 infection,http://www.ebi.ac.uk/efo/EFO_0000180
2,2-20700443,Magnesium levels,magnesium measurement,http://www.ebi.ac.uk/efo/EFO_0004845
3,3-19060906,HDL cholesterol,high density lipoprotein cholesterol measurement,http://www.ebi.ac.uk/efo/EFO_0004612
4,4-19060906,LDL cholesterol,low density lipoprotein cholesterol measurement,http://www.ebi.ac.uk/efo/EFO_0004611


In [26]:
foo = df[["trait_name", "efo_term"]].drop_duplicates().groupby(["trait_name"]).size()
foo

trait_name
&beta;2-Glycoprotein I (&beta;2-GPI) plasma levels            1
1,5-anhydroglucitol levels                                    1
17-hydroxyprogesterone (17-OHP) levels                        1
19,155 disease classifications in UK Biobank (pleiotropy)     1
3-hydroxy-1-methylpropylmercapturic acid levels in smokers    1
                                                             ..
vWF and FVIII levels                                          2
vWF levels                                                    1
vWF levels in ischaemic stroke and hyperhomocysteinaemia      1
vascular endothelial growth factor D levels                   1
visceral adipose tissue/total adipose tissue ratio            1
Length: 6339, dtype: int64

In [27]:
foo[foo > 1]

trait_name
Acute lymphoblastic leukemia (B-cell precursor)    2
Adolescent idiopathic scoliosis                    2
Advanced age-related macular degeneration          2
Advanced glycation end-product levels              2
Age of smoking initiation                          2
                                                  ..
Waist-hip ratio                                    2
Waist-to-hip ratio adjusted for body mass index    2
White blood cell count                             2
White matter hyperintensity burden                 2
vWF and FVIII levels                               2
Length: 250, dtype: int64

In [45]:
# df[df["trait_name"] == "Waist-hip ratio"]
df[df["trait_name"] == "vWF and FVIII levels"]

,trait_id,trait_name,efo_term,efo_id
1667,1667-21810271,vWF and FVIII levels,coagulation factor measurement,http://www.ebi.ac.uk/efo/EFO_0004634
5436,5436-30586737,vWF and FVIII levels,"factor VIII measurement, von Willebrand factor...","http://www.ebi.ac.uk/efo/EFO_0004630, http://w..."


Try assessing the trait-term mapping file instead

---

# Trait term mapping

In [31]:
trait_df_path = ebi_dir / "gwas_catalog_trait-mappings_r2021-06-08.tsv"
assert trait_df_path.exists()

In [36]:
df0 = pd.read_csv(trait_df_path, sep="\t")
print(df0.info())
df0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18263 entries, 0 to 18262
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Disease trait  18263 non-null  object
 1   EFO term       18263 non-null  object
 2   EFO URI        18263 non-null  object
 3   Parent term    18263 non-null  object
 4   Parent URI     18263 non-null  object
dtypes: object(5)
memory usage: 713.5+ KB
None


,Disease trait,EFO term,EFO URI,Parent term,Parent URI
0,Clubfoot,familial clubfoot with or without associated l...,http://www.orpha.net/ORDO/Orphanet_199315,Other disease,http://www.ebi.ac.uk/efo/EFO_0000408
1,Alcohol dependence or heroin dependence or met...,methamphetamine dependence,http://www.ebi.ac.uk/efo/EFO_0004701,Neurological disorder,http://www.ebi.ac.uk/efo/EFO_0000618
2,Methamphetamine dependence,methamphetamine dependence,http://www.ebi.ac.uk/efo/EFO_0004701,Neurological disorder,http://www.ebi.ac.uk/efo/EFO_0000618
3,Hypothyroidism,hypothyroidism,http://www.ebi.ac.uk/efo/EFO_0004705,Other disease,http://www.ebi.ac.uk/efo/EFO_0000408
4,Age at menopause,age at menopause,http://www.ebi.ac.uk/efo/EFO_0004704,Other measurement,http://www.ebi.ac.uk/efo/EFO_0001444
...,...,...,...,...,...
18258,Moderate or severe prolonged lymphopenia in di...,relapsing-remitting multiple sclerosis,http://www.ebi.ac.uk/efo/EFO_0003929,Immune system disorder,http://www.ebi.ac.uk/efo/EFO_0000540
18259,Severe prolonged lymphopenia in dimethyl fumar...,relapsing-remitting multiple sclerosis,http://www.ebi.ac.uk/efo/EFO_0003929,Immune system disorder,http://www.ebi.ac.uk/efo/EFO_0000540
18260,Surgical necrotising enterocolitis in extremel...,necrotizing enterocolitis,http://www.ebi.ac.uk/efo/EFO_0003928,Digestive system disorder,http://www.ebi.ac.uk/efo/EFO_0000405
18261,Cognitive performance,neuropsychological test,http://www.ebi.ac.uk/efo/EFO_0003926,Other measurement,http://www.ebi.ac.uk/efo/EFO_0001444


In [35]:
df0["Parent term"].value_counts()

Other measurement                   11677
Neurological disorder                 912
Lipid or lipoprotein measurement      705
Response to drug                      664
Other disease                         613
Other trait                           574
Cancer                                534
Biological process                    412
Cardiovascular disease                368
Cardiovascular measurement            318
Immune system disorder                309
Metabolic disorder                    295
Body measurement                      294
Digestive system disorder             216
Hematological measurement             211
Inflammatory measurement              113
Liver enzyme measurement               48
Name: Parent term, dtype: int64

In [37]:
print(df0["Disease trait"].drop_duplicates().pipe(len))

13408


In [38]:
check_mapping = (
    df0[["Disease trait", "EFO term"]].drop_duplicates().groupby("Disease trait").size()
)
check_mapping.value_counts()

1      10637
2       2050
3        494
4        113
5         40
6         17
7         12
8         11
11         9
9          6
10         4
15         2
16         2
12         1
235        1
19         1
13         1
66         1
18         1
53         1
329        1
41         1
22         1
55         1
dtype: int64

In [39]:
check_mapping[check_mapping == 329]

Disease trait
Blood protein levels    329
dtype: int64

In [41]:
check_mapping[check_mapping == 9]

Disease trait
Age-related disease endophenotypes                                 9
Alzheimer's disease or cardiovascular risk factors (pleiotropy)    9
Glucose homeostasis traits                                         9
Red blood cell traits                                              9
Serum metabolite levels                                            9
Sex hormone levels                                                 9
dtype: int64

In [40]:
df0[df0["Disease trait"] == "Blood protein levels"]

,Disease trait,EFO term,EFO URI,Parent term,Parent URI
108,Blood protein levels,SPARC-like protein 1 measurement,http://www.ebi.ac.uk/efo/EFO_0008289,Other measurement,http://www.ebi.ac.uk/efo/EFO_0001444
109,Blood protein levels,small nuclear ribonucleoprotein F measurement,http://www.ebi.ac.uk/efo/EFO_0008288,Other measurement,http://www.ebi.ac.uk/efo/EFO_0001444
110,Blood protein levels,tenascin measurement,http://www.ebi.ac.uk/efo/EFO_0008296,Other measurement,http://www.ebi.ac.uk/efo/EFO_0001444
111,Blood protein levels,T-lymphocyte surface antigen Ly-9 measurement,http://www.ebi.ac.uk/efo/EFO_0008295,Other measurement,http://www.ebi.ac.uk/efo/EFO_0001444
112,Blood protein levels,thioredoxin domain-containing protein 12 measu...,http://www.ebi.ac.uk/efo/EFO_0008298,Other measurement,http://www.ebi.ac.uk/efo/EFO_0001444
...,...,...,...,...,...
17130,Blood protein levels,galectin-3-binding protein measurement,http://www.ebi.ac.uk/efo/EFO_0010241,Other measurement,http://www.ebi.ac.uk/efo/EFO_0001444
17131,Blood protein levels,beta globulin measurement,http://www.ebi.ac.uk/efo/EFO_0010242,Other measurement,http://www.ebi.ac.uk/efo/EFO_0001444
17468,Blood protein levels,soluble transferrin receptor measurement,http://www.ebi.ac.uk/efo/EFO_0004460,Other measurement,http://www.ebi.ac.uk/efo/EFO_0001444
17870,Blood protein levels,ferritin measurement,http://www.ebi.ac.uk/efo/EFO_0004459,Other measurement,http://www.ebi.ac.uk/efo/EFO_0001444


In [43]:
df0[df0["Disease trait"] == "Age-related disease endophenotypes"]

,Disease trait,EFO term,EFO URI,Parent term,Parent URI
2991,Age-related disease endophenotypes,high density lipoprotein cholesterol measurement,http://www.ebi.ac.uk/efo/EFO_0004612,Lipid or lipoprotein measurement,http://www.ebi.ac.uk/efo/EFO_0004732
3652,Age-related disease endophenotypes,systolic blood pressure,http://www.ebi.ac.uk/efo/EFO_0006335,Other measurement,http://www.ebi.ac.uk/efo/EFO_0001444
3709,Age-related disease endophenotypes,diastolic blood pressure,http://www.ebi.ac.uk/efo/EFO_0006336,Other measurement,http://www.ebi.ac.uk/efo/EFO_0001444
6680,Age-related disease endophenotypes,ventricular rate measurement,http://www.ebi.ac.uk/efo/EFO_0007928,Cardiovascular measurement,http://www.ebi.ac.uk/efo/EFO_0004298
9159,Age-related disease endophenotypes,hematocrit,http://www.ebi.ac.uk/efo/EFO_0004348,Hematological measurement,http://www.ebi.ac.uk/efo/EFO_0004503
9206,Age-related disease endophenotypes,body mass index,http://www.ebi.ac.uk/efo/EFO_0004340,Body measurement,http://www.ebi.ac.uk/efo/EFO_0004324
16430,Age-related disease endophenotypes,triglyceride measurement,http://www.ebi.ac.uk/efo/EFO_0004530,Lipid or lipoprotein measurement,http://www.ebi.ac.uk/efo/EFO_0004529
16688,Age-related disease endophenotypes,total cholesterol measurement,http://www.ebi.ac.uk/efo/EFO_0004574,Lipid or lipoprotein measurement,http://www.ebi.ac.uk/efo/EFO_0004732
17445,Age-related disease endophenotypes,glucose measurement,http://www.ebi.ac.uk/efo/EFO_0004468,Other measurement,http://www.ebi.ac.uk/efo/EFO_0001444


---

# Cleaning

- produce the following
  - gwas-to-efo 'edge' list
- cleaning steps: 
  - remove mappings where a trait maps to more than `5` efo terms

In [47]:
SIZE_LIMIT_MULTI_MAPPING = 5

In [49]:
df1 = df0.rename(
    columns={
        "Disease trait": "gwas_name",
        "EFO term": "efo_name",
        "EFO URI": "efo_id",
        "Parent term": "parent_efo_name",
        "Parent URI": "parent_efo_id",
    }
)
print(df1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18263 entries, 0 to 18262
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   gwas_name        18263 non-null  object
 1   efo_name         18263 non-null  object
 2   efo_id           18263 non-null  object
 3   parent_efo_name  18263 non-null  object
 4   parent_efo_id    18263 non-null  object
dtypes: object(5)
memory usage: 713.5+ KB
None


In [48]:
exclude_traits = check_mapping[check_mapping > SIZE_LIMIT_MULTI_MAPPING]
print(len(exclude_traits))
exclude_traits[:5]

74


Disease trait
Age-related disease endophenotypes                                          9
Age-related diseases and mortality                                          7
Age-related diseases, mortality and associated endophenotypes              16
Alzheimer's disease or cardiovascular risk factors (pleiotropy)             9
Amyotrophic lateral sclerosis and immune-mediated diseases (pleiotropy)     7
dtype: int64

In [50]:
exclude_traits_list = list(exclude_traits.index)
exclude_traits_list[:5]

['Age-related disease endophenotypes',
 'Age-related diseases and mortality',
 'Age-related diseases, mortality and associated endophenotypes',
 "Alzheimer's disease or cardiovascular risk factors (pleiotropy)",
 'Amyotrophic lateral sclerosis and immune-mediated diseases (pleiotropy)']

In [52]:
gwas_to_efo = df1[df1["gwas_name"].apply(lambda x: x not in exclude_traits_list)]
print(gwas_to_efo.info())
gwas_to_efo.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16871 entries, 0 to 18262
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   gwas_name        16871 non-null  object
 1   efo_name         16871 non-null  object
 2   efo_id           16871 non-null  object
 3   parent_efo_name  16871 non-null  object
 4   parent_efo_id    16871 non-null  object
dtypes: object(5)
memory usage: 790.8+ KB
None


,gwas_name,efo_name,efo_id,parent_efo_name,parent_efo_id
0,Clubfoot,familial clubfoot with or without associated l...,http://www.orpha.net/ORDO/Orphanet_199315,Other disease,http://www.ebi.ac.uk/efo/EFO_0000408
1,Alcohol dependence or heroin dependence or met...,methamphetamine dependence,http://www.ebi.ac.uk/efo/EFO_0004701,Neurological disorder,http://www.ebi.ac.uk/efo/EFO_0000618
2,Methamphetamine dependence,methamphetamine dependence,http://www.ebi.ac.uk/efo/EFO_0004701,Neurological disorder,http://www.ebi.ac.uk/efo/EFO_0000618
3,Hypothyroidism,hypothyroidism,http://www.ebi.ac.uk/efo/EFO_0004705,Other disease,http://www.ebi.ac.uk/efo/EFO_0000408
4,Age at menopause,age at menopause,http://www.ebi.ac.uk/efo/EFO_0004704,Other measurement,http://www.ebi.ac.uk/efo/EFO_0001444


In [53]:
output_file = ebi_dir / "clean_gwas_to_efo.csv"
gwas_to_efo.to_csv(output_file, index=False)